# Exercise 10: Color Space Transformation

https://docs.opencv.org/4.6.0/df/d9d/tutorial_py_colorspaces.html

In this exercise you will:
- Learn how to convert images from one color space to another, e.g. BGR $\leftrightarrow$ Gray and BGR $\leftrightarrow$ HSV.
- Learn how to use the OpenCV functions [cv.cvtColor()](https://docs.opencv.org/4.6.0/d8/d01/group__imgproc__color__conversions.html#ga397ae87e1288a81d2363b61574eb8cab) and [cv.inRange()](https://docs.opencv.org/4.6.0/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981).
- Create an application to extract, and track, a colored object in a video.

As a first step, let's import the python modules we need.

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

## Changing Color Space

Sometimes it's easier to work with colors in another color space, e.g. BGR (Blue, Red, green) or Grayscale. There are more than 150 color space conversion methods available in OpenCV, but we will only look at the two most widely used ones: BGR $\leftrightarrow$ Gray and BGR $\leftrightarrow$ HSV (Hue, Saturation, Value).

For color conversion, we use the function `cv.cvtColor(input_image, flag)` where `flag` determines the type of conversion.

For BGR $\rightarrow$ Gray conversion, we use the flag [cv.COLOR_BGR2GRAY](https://docs.opencv.org/4.6.0/d8/d01/group__imgproc__color__conversions.html#gga4e0972be5de079fed4e3a10e24ef5ef0a353a4b8db9040165db4dacb5bcefb6ea). Similarly for BGR $\rightarrow$ HSV, we use the flag [cv.COLOR_BGR2HSV](https://docs.opencv.org/4.6.0/d8/d01/group__imgproc__color__conversions.html#gga4e0972be5de079fed4e3a10e24ef5ef0aa4a7f0ecf2e94150699e48c79139ee12). To get other flags, i.e. [color conversion codes](https://docs.opencv.org/4.6.0/d8/d01/group__imgproc__color__conversions.html#ga4e0972be5de079fed4e3a10e24ef5ef0), such as `cv.COLOR_BGR2RGB`, just run the following Python code.

In [2]:
flags = [i for i in dir(cv) if i.startswith('COLOR_')]
print(flags)

['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGRA', 'COLOR_BAYER_BG2BGR_EA', 'COLOR_BAYER_BG2BGR_VNG', 'COLOR_BAYER_BG2GRAY', 'COLOR_BAYER_BG2RGB', 'COLOR_BAYER_BG2RGBA', 'COLOR_BAYER_BG2RGB_EA', 'COLOR_BAYER_BG2RGB_VNG', 'COLOR_BAYER_BGGR2BGR', 'COLOR_BAYER_BGGR2BGRA', 'COLOR_BAYER_BGGR2BGR_EA', 'COLOR_BAYER_BGGR2BGR_VNG', 'COLOR_BAYER_BGGR2GRAY', 'COLOR_BAYER_BGGR2RGB', 'COLOR_BAYER_BGGR2RGBA', 'COLOR_BAYER_BGGR2RGB_EA', 'COLOR_BAYER_BGGR2RGB_VNG', 'COLOR_BAYER_GB2BGR', 'COLOR_BAYER_GB2BGRA', 'COLOR_BAYER_GB2BGR_EA', 'COLOR_BAYER_GB2BGR_VNG', 'COLOR_BAYER_GB2GRAY', 'COLOR_BAYER_GB2RGB', 'COLOR_BAYER_GB2RGBA', 'COLOR_BAYER_GB2RGB_EA', 'COLOR_BAYER_GB2RGB_VNG', 'COLOR_BAYER_GBRG2BGR', 'COLOR_BAYER_GBRG2BGRA', 'COLOR_BAYER_GBRG2BGR_EA', 'COLOR_BAYER_GBRG2BGR_VNG', 'COLOR_BAYER_GBRG2GRAY', 'COLOR_BAYER_GBRG2RGB', 'COLOR_BAYER_GBRG2RGBA', 'COLOR_BAYER_GBRG2RGB_EA', 'COLOR_BAYER_GBRG2RGB_VNG', 'COLOR_BAYER_GR2BGR', 'COLOR_BAYER_GR2BGRA', 'COLOR_BAYER_GR2BGR_EA', 'COLOR_BAYER_GR2BGR_VNG', 'COLOR_

## HSV (Hue, Saturation, Value)

<p>

<img width="500" height="200" src="../notebook_images/rgb_hsv.jpg" style="padding: 10px; float: right;">

[HSL and HSV] (https://en.wikipedia.org/wiki/HSL_and_HSV) are two similar color spaces, where we will focus on HSV, which is modelled after how we perceive colors when they are exposed to various intensities of white light. HSV stands for Hue, Saturation and Value:
- **Hue** is the percieved color (red, green, blue, yellow or a combinations of these).
- **Saturation** is the "colorfulness" of a color, i.e. how "deep" the color is.
- **Value** is the brightness of a color, i.e. imagine shining a torch with an adjustable intensity on a colored object.

The ranges for Hue, Saturation and Value are as follows:
- **Hue**: $[0, 179]$
- **Saturation**: $[0, 255]$
- **Value**: $[0, 255]$

Other image processing software might use different scales, so if you are comparing OpenCV's values with other software's values, you need to normalize these three ranges, e.g. to $[0, 1]$.
</p>

## Object Tracking

<p>

<img width="500" height="150" src="../notebook_images/color_mask.jpg" style="padding: 10px; float: right;">

Now that we know how to convert a BGR image to HSV, we can use this to extract a colored object. In HSV, it is easier to represent a color than in BGR color space. Let's build an application using a simple color-based object tracking method. In our application, we will extract a blue colored object from our cameras's video stream, in order to track it from frame to frame.

Here's the method we will implement:

1. Capture each frame of the camera's video stream.
2. Convert the frame from BGR to HSV color space.
3. Threshold the HSV frame for a range of blue color.
4. Extract the blue object(s).

We can use the OpenCV function [cv.inRange()](https://docs.opencv.org/4.6.0/d2/de8/group__core__array.html#ga48af0ab51e36436c5d04340e036ce981) for the thresholding (step 3) above. The `dst = inRange(src, lowerb, upperb)` function takes an input image `scr`, together with a lower- and upper bound (`lowerb` and `upperb`) as inputs, and returns an array, in which the pixel values are set to:
- $255$ if the the pixel in the input image `src` falls within the lower- and upper bound (`lowerb` and `upperb`).
- $0$ otherwise.
</p>

**Note! You need to hold up a blue object infront of your camera. OpenCV will create three windows that might be minimized in your tray. You need to select one of these windows and press the escape <esc> key on your keyboard to close the windows (otherwise the cell won't finish executing).**

In [3]:
# Create a VideoCapture object
# to capture our camera's video stream.
cap = cv.VideoCapture(0)

while(True):
    # Capture a frame with our camera.
    _, frame = cap.read()

    # Convert the frame from BGR to HSV color space.
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    
    # Define a low and high threshold of blue color in HSV.
    lower_blue = np.array([110,50,50])
    upper_blue = np.array([130,255,255])
    
    # Threshold HSV frame to get only the desired blue colors
    # defined with the two thresholds above. The blue pixels
    # within the two thresholds will be set to 255 in our mask,
    # and all other pixels will be set to 0 in our mask.
    mask = cv.inRange(hsv, lower_blue, upper_blue)
    
    # Bitwise AND the frame with itself and the mask.
    # The result is an image with all zeros, except for the
    # pixels that agree with our mask (which are kept intact).
    res = cv.bitwise_and(frame, frame, mask=mask)
    
    # Let's display the original frame, the mask frame
    # and the final frame obtained when applying the mask.
    # We will show the three frames in their own window.
    cv.imshow('frame', frame)
    cv.imshow('mask', mask)
    cv.imshow('res', res)
    
    # Check for any keyboard events.
    k = cv.waitKey(5) & 0xFF

    # If the escape key is pressed, stop capturing and
    # processing video frames (i.e. exit out of the 'while' loop).
    if k == 27:
        break

# Finally, release the VideoCapture, and destroy any windows we have created.
cap.release()
cv.destroyAllWindows()

## How do we find the HSV values (from BGR values) to track?

We can use the function `cv.cvtColor()` to find the HSV values to use from a BGR (or RGB) color. Instead of passing an image as the first argument, we can pass the BGR (or RGB) values we want to convert to HSV values. For example, to find the HSV values for Green, i.e. $[b,g,r] = [0,255,0]$, we can run the code below.

In [4]:
# Create an array with shape (1,1,3)
# that contains the BGR color we want
# to convet to a HSB color.
bgr_green = np.uint8([[[0,255,0]]])

# Use 'cv.cvtColor()' with 'cv.COLOR_BGR2HSV'
# to convert the BGR color to a HSV color.
hsv_green = cv.cvtColor(bgr_green, cv.COLOR_BGR2HSV)

# Print the HSV color.
print(f'The BGR color {bgr_green} is equivalent to the HSV color {hsv_green}')

# Print possible HSV color bounds to use in the function cv.inRange() in our tracking application above.
H, S, V = hsv_green[0,0,:]
lower_bound = np.array([H-10,S,V]) 
upper_bound = np.array([H+10,S,V]) 
print(f'Try the following thresholds: mask = cv.inRange(hsv, {lower_bound}, {upper_bound})')

The BGR color [[[  0 255   0]]] is equivalent to the HSV color [[[ 60 255 255]]]
Try the following thresholds: mask = cv.inRange(hsv, [ 50 255 255], [ 70 255 255])
